In [ ]:
import sys
sys.path.append("../mlexplainer")

In [2]:
from shap_explainer import BinaryMLExplainer

import numpy as np
import pandas as pd

In [3]:
dataframe = pd.read_csv('../tests/resources/samples/df_nba_rookie.csv')
dataframe.shape[0]

1340

In [4]:
dataframe["is_ok_3pts"] = dataframe["3PA"].apply(lambda u: "2 +" if u > 2 else "Entre 0 et 1" if u > 1 else "Moins de 1")

dataframe["is_ok_3pts"] = dataframe["is_ok_3pts"].astype("category")
dataframe["lebron"] = "lebron"

dataframe["is_ok_3pts"].value_counts()

is_ok_3pts
Moins de 1      972
Entre 0 et 1    192
2 +             176
Name: count, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

dataframe['3P%_2'] = dataframe['3P%'].apply(lambda u: np.nan if u<15 else u)
del dataframe["Name"]

In [6]:
target = 'TARGET_5Yrs'
selected_features = [c for c in dataframe if (c != target) and c not in ["Name"]]

### Séparation en X et y ###
X, y = dataframe, dataframe[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
from lightgbm import LGBMClassifier

model = LGBMClassifier()
fitted_model = model.fit(X_train[selected_features], y_train)

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: lebron: object

In [9]:
binary_explainer = BinaryMLExplainer(x_train=X_train[selected_features], 
                   y_train=y_train, 
                   x_test=X_test[selected_features], 
                   y_test=y_test, 
                   model=fitted_model, 
                   features=selected_features)

binary_explainer.categorical_features

['is_ok_3pts']

In [10]:
binary_explainer.numerical_features

['GP',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3P Made',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 '3P%_2']

In [11]:
binary_explainer.string_features

[]